Lab 3: Language Modeling
=============
In this problem set, your objective is to train a language model, evaluate it and explore how it can be used for language generation. Towards that end you will:

- Train an n-gram language model.
- Use that language model to generate representative sentences.
- Study the effect of training data size, and language model complexity (n-gram size), on the modeling capacity of a language model.

- **For this assignment, submit ```lab3.py``` on Gradescope.**
- In order to test the lab you can run ```python run_tests.py``` or ```python run_tests.py -j``` (more detailed information)
- In order to install the correct dependencies you can run ```pip install -r requirements.txt```

Total points: 90 points

# 0. Setup

In order to develop this assignment, you will need [python 3.6](https://www.python.org/downloads/) and the following libraries. Most if not all of these are part of [anaconda](https://www.continuum.io/downloads), so a good starting point would be to install that.

- [jupyter](http://jupyter.readthedocs.org/en/latest/install.html)
- [nosetests](https://nose.readthedocs.org/en/latest/)
- [nltk](https://www.nltk.org)

Here is some help on installing packages in python: https://packaging.python.org/installing/. You can use ```pip --user``` to install locally without sudo. We have also provided a requirements.txt file with the correct packages and their respective versions, so you can also run ```pip install -r requirements.txt``` to install the correct dependencies

In [1]:
import sys
from importlib import reload
from collections import defaultdict
import lab3

In [2]:
print('My Python version')

print('python: {}'.format(sys.version))

My Python version
python: 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]


In [4]:
import nose
import nltk

In [5]:
print('My library versions')

print('nose: {}'.format(nose.__version__))
print('nltk: {}'.format(nltk.__version__))

My library versions
nose: 1.3.7
nltk: 3.6.1


To test whether your libraries are the right version, run:

`nosetests tests/test_environment.py`

In [5]:
! nosetests tests/test_environment.py

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# 1. Training a language model

Let us first train a 3-gram language model. We need a monolingual corpus, which we will get using nltk.

Total: 40 points

Let us first extract from nltk's reuters corpus, 2 corpora in 2 different domains (here, subject areas), the food industry and the natural resources industry.

In [8]:
import nltk

food = ['barley', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copra-cake', 'grain', 'groundnut', 'groundnut-oil', 'potato', 'soy-meal', 'soy-oil', 'soybean', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'veg-oil', 'wheat']
natural_resources = ['alum', 'fuel', 'gas', 'gold', 'iron-steel', 'lead', 'nat-gas', 'palladium', 'propane', 'tin', 'zinc']
corpus = nltk.corpus.reuters
food_corpus = corpus.raw(categories=food)
natr_corpus = corpus.raw(categories=natural_resources)

In [15]:
print(type(food_corpus))
print(food_corpus[:1000])
food_corpus[:1000]

<class 'str'>
CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS
  A survey of 19 provinces and seven cities
  showed vermin consume between seven and 12 pct of China's grain
  stocks, the China Daily said.
      It also said that each year 1.575 mln tonnes, or 25 pct, of
  China's fruit output are left to rot, and 2.1 mln tonnes, or up
  to 30 pct, of its vegetables. The paper blamed the waste on
  inadequate storage and bad preservation methods.
      It said the government had launched a national programme to
  reduce waste, calling for improved technology in storage and
  preservation, and greater production of additives. The paper
  gave no further details.
  

THAI TRADE DEFICIT WIDENS IN FIRST QUARTER
  Thailand's trade deficit widened to 4.5
  billion baht in the first quarter of 1987 from 2.1 billion a
  year ago, the Business Economics Department said.
      It said Janunary/March imports rose to 65.1 billion baht
  from 58.7 billion. Thailand's improved business climate this


"CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS\n  A survey of 19 provinces and seven cities\n  showed vermin consume between seven and 12 pct of China's grain\n  stocks, the China Daily said.\n      It also said that each year 1.575 mln tonnes, or 25 pct, of\n  China's fruit output are left to rot, and 2.1 mln tonnes, or up\n  to 30 pct, of its vegetables. The paper blamed the waste on\n  inadequate storage and bad preservation methods.\n      It said the government had launched a national programme to\n  reduce waste, calling for improved technology in storage and\n  preservation, and greater production of additives. The paper\n  gave no further details.\n  \n\nTHAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n  Thailand's trade deficit widened to 4.5\n  billion baht in the first quarter of 1987 from 2.1 billion a\n  year ago, the Business Economics Department said.\n      It said Janunary/March imports rose to 65.1 billion baht\n  from 58.7 billion. Thailand's improved business climate

## Tokenization

Your first task is to tokenize the raw text into a list of sentences, which are in turn a list of words. No need for any other kind of preprocessing such as lowercasing.

- **Deliverable 1.1**: Complete the function `lab3.tokenize`. (5 points)
- **Test**: `tests/test_visible.py:test_d1_1_tk`

In [7]:
reload(lab3);

In [8]:
food_corpus_tk = lab3.tokenize_corpus(food_corpus)
natr_corpus_tk = lab3.tokenize_corpus(natr_corpus)

## Padding

Your second task is to pad your sentences with the start-of-sentence symbol `'<s>'` and end-of-sentence symbol `'</s>'`. These symbols are necessary to model the probability of words that usually start a sentence and those that usually end a sentence.

- **Deliverable 1.2**: Complete the function `lab3.pad`. (5 points)
- **Test**: `tests/test_visible.py:test_d1_2_pad`

In [9]:
reload(lab3);

In [10]:
food_corpus_tk_pd = lab3.pad_corpus(food_corpus_tk)
natr_corpus_tk_pd = lab3.pad_corpus(natr_corpus_tk)

## Train-Test Split

Your third task is to split the corpora into train, for training the language model, and test, for testing the language model. We will go with the traditional 80% (train), 20% (test) split. The first `floor(0.8*num_of_tokens)` should constitute the training corpus, and the rest should constitute the test corpus.

- **Deliverable 1.3**: Complete the function `lab3.split_corpus`. (5 points)
- **Test**: `tests/test_visible.py:test_d1_3_spc`

In [11]:
reload(lab3);

In [12]:
food_corpus_tr, food_corpus_te = lab3.split_corpus(food_corpus_tk_pd)
natr_corpus_tr, natr_corpus_te = lab3.split_corpus(natr_corpus_tk_pd)

## Splitting into n-grams

Your fourth task is to count n-grams in the text up to a specific order.

- **Deliverable 1.4**: Complete the function `lab3.count_ngrams`. (20 points)
- **Test**: `tests/test_visible.py:test_d1_4_cn`

In [13]:
reload(lab3);

In [14]:
food_ngrams, food_vocab = lab3.count_ngrams(food_corpus_tr, 3)
natr_ngrams, natr_vocab = lab3.count_ngrams(natr_corpus_tr, 3)

## Estimating n-gram probability

Your last task in this part of the problem set is to estimate the n-gram probabilities p(w_i|w_{i-n+1}, w_{i-n+2}, .., w_{i-1}), with no smoothing. For the purposes of this exercise we will use the maximum likelihood estimate and perform no smoothing. 

- **Deliverable 1.5**: Complete the function `lab3.estimate`. (5 points)
- **Test**: `tests/test_visible.py:test_d1_5_es`

In [15]:
reload(lab3);

In [16]:
print(lab3.estimate(food_ngrams, ['palm'], ['producer', 'of']))
print(lab3.estimate(natr_ngrams, ['basis'], ['tested', 'the']))

0.25
0.5


Application: the speech recognition task takes human voice as its input and outputs text. If the pronunciation of two words are similar, Language Model can help decide which word to choose! 

In [17]:
print(food_ngrams[('there', 'is', 'no')])
print(food_ngrams[('their', 'is', 'no')])

11
0


Given the count of 'there is no' and 'their is no', which word ('there' or 'their') is more likely to be taken as the output? 

Language Model is not only helpful in speech recogition, but text generation (*e.g.*, machine translation, summarization, image captioning), spelling correction and so on. 

## Training a language model

Now we will combine everything together and train our language model! One way to see what the language model has learned is to see the sentences it can generate.

For the sake of simplicity, and for the purposes of later parts in this problem set, we use nltk's lm module to train a language model.

In [18]:
from nltk.lm import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
size_ngram = 3

food_train, food_vocab = padded_everygram_pipeline(size_ngram, food_corpus_tk[:int(0.8*len(food_corpus_tk))])
natr_train, natr_vocab = padded_everygram_pipeline(size_ngram, natr_corpus_tk[:int(0.8*len(natr_corpus_tk))])

food_test = sum([['<s>'] + x + ['</s>'] for x in food_corpus_tk[int(0.8*len(food_corpus_tk)):]],[])
natr_test = sum([['<s>'] + x + ['</s>'] for x in natr_corpus_tk[int(0.8*len(natr_corpus_tk)):]],[])

food_lm = Laplace(size_ngram)
natr_lm = Laplace(size_ngram)

food_lm.fit(food_train, food_vocab)
natr_lm.fit(natr_train, natr_vocab)

Now let's ask our language model to generate a sentence. 

In [19]:
# This might take some time
n_words = 10
print(food_lm.generate(n_words, random_seed=3))  # random_seed makes the random sampling part of generation reproducible. 
print(natr_lm.generate(n_words, random_seed=3))

['<s>', 'Coffee', 'does', 'not', 'know', 'how', 'much', 'they', 'planted', '.']
['<s>', 'Currently', ',', 'there', 'was', 'a', '15.3', 'pct', 'increase', ',']


# 2. Evaluating a language model

Next, we evaluate our language models using the perplexity measure, and draw conclusions on how a change of domains (here, subject areas) can affect the performance of a language model. Perplexity measures the language model capacity at predicting sentences in a test corpus.

Total: 10 points

- **Deliverable 2.1**: Complete the function `lab3.get_perplexity`. (10 points)
- **Test**: `tests/test_visible.py:test_d2_1_gp`

In [20]:
reload(lab3);

In [21]:
# This might take some time
print(lab3.get_perplexity(food_lm, food_test[:5000]))
print(lab3.get_perplexity(food_lm, natr_test[:5000]))
print(lab3.get_perplexity(natr_lm, natr_test[:5000]))
print(lab3.get_perplexity(natr_lm, food_test[:5000]))

8396.70521548381
8546.64191689775
5428.123517045601
5490.520376861629


- What observations can you make on the results? Is the domain shift affecting the performance of the language model? What are possible explanations?


**Your Observation**: 

# 3. Data size and model complexity

Let us now see how the size of the training data and the complexity of the model we choose affects the quality of our language model.

Total: 40 points

For this part we'd like to see the difference between a 2-gram model and a 3-gram model. Typically, with a larger n, the n-gram model gives us more information about the word sequence and has lower perplexity. 

For testing, we'll only be considering 5% instead of 20% of the test data for running time purposes. 

- **Deliverable 3.1**: Complete the function `lab3.vary_ngram`. (40 points)
- **Test**: `tests/test_visible.py:test_d3_1_vary`

In [22]:
reload(lab3);

In [23]:
n_gram_orders = [2, 3]

train_corpus = natr_corpus_tk[:int(0.8*len(natr_corpus_tk))]
test_corpus = natr_corpus_tk[int(0.8*len(natr_corpus_tk)): int(0.85*len(natr_corpus_tk))]

results = lab3.vary_ngram(train_corpus, test_corpus, n_gram_orders)

print(results)

2
3
{2: 5393.67203181578, 3: 5424.76053819262}


However, we notice that the 3-gram language model actually performs worse than the 2-gram language model. This is due to the small size of the training corpus. A 3-gram language model is actually too complex of a model for a small training size. If our training data was larger, we would be seeing the opposite. If we trained 1-gram, 2-gram, and 3-gram models on 38 million words from the Wall Street Journal, we will get perplexity of 962, 170, 109 respectively on a test set of 1.5 million words. 

Now let's see a few examples of top frequent n-gram examples. Let's start with unigram. 

In [24]:
natr_ngrams, natr_vocab = lab3.count_ngrams(natr_corpus_tr, 3)

top_ngram = []
count = 0
for i in sorted(natr_ngrams.items(), key=lambda x: x[1], reverse=True):
    if len(i[0]) == 1:
        top_ngram.append(i[0])
        count += 1
    if count >=20:
        break
print(top_ngram)

[(',',), ('the',), ('<s>',), ('</s>',), ('.',), ('of',), ('to',), ('and',), ('said',), ('in',), ('a',), ('for',), ('The',), ('from',), ('pct',), ('mln',), ('at',), ('on',), ("'s",), ('is',)]


Do you think unigram captures any grammatical information? How well do you think unigram captures the language information? 

Now let's see bigram and trigram. 

In [25]:
top_ngram = []
count = 0
for i in sorted(natr_ngrams.items(), key=lambda x: x[1], reverse=True):
    if len(i[0]) == 2:
        top_ngram.append(i[0])
        count += 1
    if count >=20:
        break
print(top_ngram)

top_ngram = []
count = 0
for i in sorted(natr_ngrams.items(), key=lambda x: x[1], reverse=True):
    if len(i[0]) == 3:
        top_ngram.append(i[0])
        count += 1
    if count >=20:
        break
print(top_ngram)

[('.', '</s>'), ('said', '.'), ('<s>', 'The'), ('in', 'the'), ('of', 'the'), ('&', 'lt'), ('lt', ';'), (',', 'the'), ('said', 'it'), ('said', 'the'), ('<s>', '``'), (',', "''"), (',', 'which'), ('to', 'the'), ('for', 'the'), (',', 'a'), ('on', 'the'), (',', 'and'), ('mln', 'dlrs'), ('<s>', 'It')]
[('said', '.', '</s>'), ('&', 'lt', ';'), ('.', "''", '</s>'), ('<s>', 'The', 'company'), ('<s>', 'It', 'said'), ('he', 'said', '.'), ('ounces', 'of', 'gold'), ('year', '.', '</s>'), ('The', 'company', 'said'), ('...', '...', '...'), ('added', '.', '</s>'), ('oil', 'and', 'gas'), (',', 'it', 'said'), ('pct', '.', '</s>'), (',', "''", 'he'), (',', 'he', 'said'), ('it', 'said', '.'), ('sources', 'said', '.'), ('is', 'expected', 'to'), ('<s>', 'He', 'said')]


Compared with unigram, bigram and trigram can capture more information. 
Bigram language model can already capture some of the grammatical information, such as 'in the', 'of the'. However, the power of bigram is still limited. 
The trigram can output more adequate short phrases such as 'ounces of gold', 'The company said', 'oil and gas'. 

Therefore, typically the n-gram model with a larger n contains more information about the word sequence and thus, has lower perplexity. However, the tradeoff is the computational efficiency and memory. 